<a href="https://colab.research.google.com/github/LEEYOUNHO-HUFS/HUFS-Infinite-challenge-portfolio/blob/main/Colab%20%ED%95%9C%EA%B5%AD%EC%96%B4%20%ED%98%95%ED%83%9C%EC%86%8C%20%EB%B6%84%EC%84%9D%20%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [ ]:
# konlpy(한국어형태소분석기)를 실행하기
import konlpy

ModuleNotFoundError: ignored

In [ ]:
# konlpy 설치하기
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 8.8MB/s 
     |████████████████████████████████| 460kB 35.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
# 실습을 위한 역대 대통령 취임사: 노무현, 이명박, 박근혜, 문재인
## https://drive.google.com/open?id=1VEdcJErjAjj2ul5gnS0-IgKrpGy0NEAK

rawdata_downloaded = drive.CreateFile({'id': '1ObeV7_4IoaogmaIbK1N1Eum38lLL0Pfr'})
rawdata_downloaded.GetContentFile('rawdata.txt')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('rawdata.txt', sep="\t")

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,문장번호,발화자,본문,Unnamed: 3
0,1,유재석,"자, 올 한해는 정말 행복한 일이 가득하면 좋겠는데, 그래서 저희가 오랫동안 준비해...",NaN
1,2,유재석,"오늘 그 특집을, 저희가 좀 하려고 하는데요.",NaN
2,3,유재석,"예, 근데 이거를 살짝 듣기로도 야 어떻게 이런 생각을 하셨지? 하는 기발한 그런 ...",NaN
3,4,유재석,"그래서 저희가 이것을 가지고 실제로 저희 실생활에 이런 법안들이 정해질 수 있을지,...",NaN
4,5,양세형,"근데 이제 또 그 법을 만드는 사람은 따로 있잖아요, 법을 제정하는 사람도 따로 있고…",NaN
...,...,...,...,...
1286,1287,양세형,"그 숙제를, 지금 많은 숙제를 껴안게 됐는데,",NaN
1287,1288,정준하,껴안게가 아니라 떠안게.,NaN
1288,1289,양세형,"떠안게 되셨고, 내일부터 숙제를 잘 풀어주시기 바라겠습니다, 네 양세형이었습니다.",NaN
1289,1290,유재석,"과연, 이 중에 어떤 법안이 실제 발의가 될 지, 또 여기 계신 국회의원 분들과 많...",NaN


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    분류 = row[1] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "형태소" 변수의 저장 내용 확인
형태소

[('고맙습니다', 'Adjective'), ('!', 'Punctuation')]

In [ ]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,고맙습니다,Adjective,유재석,1
1,!,Punctuation,유재석,1


In [ ]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[       0            1 type  count
 0      자         Noun  유재석      1
 1      ,  Punctuation  유재석      1
 2      올         Verb  유재석      1
 3     한해         Noun  유재석      1
 4      는         Josa  유재석      1
 5     정말         Noun  유재석      1
 6    행복한    Adjective  유재석      1
 7     일이     Modifier  유재석      1
 8   가득하면    Adjective  유재석      1
 9   좋겠는데    Adjective  유재석      1
 10     ,  Punctuation  유재석      1
 11   그래서       Adverb  유재석      1
 12    저희         Noun  유재석      1
 13     가         Josa  유재석      1
 14  오랫동안       Adverb  유재석      1
 15    준비         Noun  유재석      1
 16    해온         Verb  유재석      1
 17    특집         Noun  유재석      1
 18     이         Josa  유재석      1
 19  있습니다    Adjective  유재석      1
 20     .  Punctuation  유재석      1,        0            1 type  count
 0     오늘         Noun  유재석      1
 1      그         Noun  유재석      1
 2     특집         Noun  유재석      1
 3      을         Josa  유재석      1
 4      ,  Punctuation  유재석      1
 5     저희         No

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,자,Noun,유재석,1
1,",",Punctuation,유재석,1
2,올,Verb,유재석,1
3,한해,Noun,유재석,1
4,는,Josa,유재석,1
...,...,...,...,...
42,들어갔으면,Verb,유재석,1
43,좋겠습니다,Adjective,유재석,1
44,.,Punctuation,유재석,1
0,고맙습니다,Adjective,유재석,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,자,Noun,유재석,1
1,",",Punctuation,유재석,1
2,올,Verb,유재석,1
3,한해,Noun,유재석,1
4,는,Josa,유재석,1
...,...,...,...,...
42,들어갔으면,Verb,유재석,1
43,좋겠습니다,Adjective,유재석,1
44,.,Punctuation,유재석,1
0,고맙습니다,Adjective,유재석,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소   품사           분류 
!     Punctuation  모두      3
                   박명수    17
                   시민      2
                   양세형    12
                   유재석    40
                          ..
힘들고   Adjective    유재석     1
힘들어요  Adjective    시민      1
                   이정미     1
힘듭니다  Adjective    이정미     1
힘쓰는   Verb         정준하     1
Name: 카운트, Length: 7677, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소  품사          분류      
!    Punctuation 모두     3
                 박명수   17
                 시민     2
                 양세형   12
                 유재석   40
...                   ...
힘들고  Adjective   유재석    1
힘들어요 Adjective   시민     1
                 이정미    1
힘듭니다 Adjective   이정미    1
힘쓰는  Verb        정준하    1

[7677 rows x 1 columns]

In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')